In [1]:
%load_ext autoreload
%autoreload 2

# JSON Agent

This notebook showcases an agent designed to interact with large JSON/dict objects. This is useful when you want to answer questions about a JSON blob that's too large to fit in the context window of an LLM. The agent is able to iteratively explore the blob to find what it needs to answer the user's question.

In the below example, we are using the OpenAPI spec for the OpenAI API, which you can find [here](https://github.com/openai/openai-openapi/blob/master/openapi.yaml).

We will use the JSON agent to answer some questions about the API spec.

## Initialization

In [2]:
import os
import yaml

from langchain.agents import (
    create_json_agent,
    AgentExecutor
)
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [3]:
# agent_executor = create_python_agent(
#     llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
#     tool=tool,
#     verbose=True,
#     agent_type=AgentType.OPENAI_FUNCTIONS
# )

In [4]:
# data['paths'].keys()

In [5]:
# with open("openai_openapi.yml") as f:
#     data = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
data = {
    'contacts':
    {
      "name": "John Doe",
      "age": 30,
      "email": "johndoe@example.com",
      "address": 
        {
            "main": {
            "street": "123 Main St",
            "city": "New York",
            "state": "NY",
            "zipcode": "10001",
            "phone numbers":
                [
                    '123456',
                    '654321'
                ]
          },
            "secondary": {
            "street": "456 Main St",
            "city": "New York",
            "state": "NY",
            "zipcode": "10001"  
            },
        },
      "phone_numbers": [
        {
          "type": "home",
          "number": "555-1234"
        },
        {
          "type": "work",
          "number": "555-5678"
        }
      ],
      "friends": [
        {
          "name": "Jane Smith",
          "age": 28,
          "email": "janesmith@example.com"
        },
        {
          "name": "Tom Johnson",
          "age": 32,
          "email": "tomjohnson@example.com"
        }
      ],
    }
}

In [7]:
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

In [8]:
# json_toolkit.get_tools()[0]("data['paths']")

## Example: getting the required POST parameters for a request

In [9]:
json_agent_executor = create_json_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    toolkit=json_toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

In [10]:
data['contacts']

{'name': 'John Doe',
 'age': 30,
 'email': 'johndoe@example.com',
 'address': {'main': {'street': '123 Main St',
   'city': 'New York',
   'state': 'NY',
   'zipcode': '10001',
   'phone numbers': ['123456', '654321']},
  'secondary': {'street': '456 Main St',
   'city': 'New York',
   'state': 'NY',
   'zipcode': '10001'}},
 'phone_numbers': [{'type': 'home', 'number': '555-1234'},
  {'type': 'work', 'number': '555-5678'}],
 'friends': [{'name': 'Jane Smith',
   'age': 28,
   'email': 'janesmith@example.com'},
  {'name': 'Tom Johnson', 'age': 32, 'email': 'tomjohnson@example.com'}]}

In [11]:
json_agent_executor.run("What are John Doe's main address and what are his phone numbers?")

Error in on_chain_start callback: 'name'



Invoking: `json_spec_list_keys` with `data`


['contacts']
Invoking: `json_spec_list_keys` with `data['contacts'][0]`


KeyError(0)
Invoking: `json_spec_list_keys` with `data['contacts']`


['name', 'age', 'email', 'address', 'phone_numbers', 'friends']
Invoking: `json_spec_get_value` with `data['contacts'][0]['address']`


KeyError(0)
Invoking: `json_spec_get_value` with `data['contacts'][0]['phone_numbers']`


KeyError(0)
Invoking: `json_spec_get_value` with `data['contacts'][0]['name']`


KeyError(0)
Invoking: `json_spec_get_value` with `data['contacts'][0]['phone_numbers'][0]`


KeyError(0)
Invoking: `json_spec_get_value` with `data['contacts'][0]['phone_numbers']`


KeyError(0)
Invoking: `json_spec_get_value` with `data['contacts'][0]['phone_numbers']`


KeyError(0)
Invoking: `json_spec_get_value` with `data['contacts'][0]['phone_numbers']`


KeyError(0)
Invoking: `json_spec_get_value` with `data['contacts'][0]['phone_numbers']`


KeyError(0)
Invoking: `json_spec_get_value` with 

'Agent stopped due to iteration limit or time limit.'

In [12]:
json_agent_executor.run("What are the required parameters in the request body to the /completions endpoint?")

Error in on_chain_start callback: 'name'



Invoking: `json_spec_list_keys` with `data['paths']['/completions']['post']['requestBody']['content']['application/json']['schema']['required']`


KeyError('paths')
Invoking: `json_spec_list_keys` with `data['components']['schemas']['CompletionsRequest']['required']`


KeyError('components')I don't know the required parameters in the request body to the /completions endpoint.

> Finished chain.


"I don't know the required parameters in the request body to the /completions endpoint."

In [13]:
json_agent_executor = create_json_agent(
#     llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    llm=OpenAI(),
    toolkit=json_toolkit,
    verbose=True)

In [14]:
json_agent_executor.run("What are the required parameters in the request body to the /completions endpoint?")

Error in on_chain_start callback: 'name'


Action: json_spec_list_keys
Action Input: data
Observation: ['contacts']
Thought: The contacts key looks promising, I should see what's inside
Action: json_spec_list_keys
Action Input: data["contacts"][0]
Observation: KeyError(0)
Thought: I need to go back and see what other keys are present
Action: json_spec_list_keys
Action Input: data
Observation: ['contacts']
Thought:


KeyboardInterrupt

